In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

In [3]:
path_to_file = r"C:\Users\relee\OneDrive\Desktop\science-textbook-grade-5.txt"

In [5]:
try:
    with open(path_to_file, 'r', encoding='utf-8') as f:
        text = f.read()
except UnicodeDecodeError:
    with open(path_to_file, 'r', encoding='latin-1') as f:
        text = f.read()

print(f'Length of text: {len(text)} characters')

vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

Length of text: 191976 characters
112 unique characters


In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


In [9]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

seq_length = 100
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())



C
H
A
P
T
E
R
 
1
b'\x0cCHAPTER 1\n\nStructure of Living Things . . . . . . . . . . . . . . . . . . . . . iv\nLesson 1\n\nCells .'
b' . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2\n\nLesson 2 From Cells to Or'
b'ganisms . . . . . . . . . . . . . . . . . . . 6\nLesson 3 Diversity of Organisms . . . . . . . . . . .'
b' . . . . . . . . . .10\nVocabulary Review . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . '
b'. . . .18\nCHAPTER 2\n\nPlant Structure and Functions . . . . . . . . . . . . . . . . 20\nLesson 1\n\nVascu'


In [11]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())


Input : b'\x0cCHAPTER 1\n\nStructure of Living Things . . . . . . . . . . . . . . . . . . . . . iv\nLesson 1\n\nCells '
Target: b'CHAPTER 1\n\nStructure of Living Things . . . . . . . . . . . . . . . . . . . . . iv\nLesson 1\n\nCells .'


In [13]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset


<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [15]:
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024


In [17]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        x, new_states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        return (x, new_states) if return_state else x

In [19]:
model = MyModel(vocab_size, embedding_dim, rnn_units)

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 113)


In [23]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, 100, 256)              │          28,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ((64, 100, 1024), (64,      │       3,938,304 │
│                                      │ 1024))                      │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, 100, 113)              │         115,825 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,083,057 (15.58 MB)

 Trainable params: 4,083,057 (15.58 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([ 86, 109,  46,  36,  17, 100,  65,  18,  13,  71,  30,  49,  36,
        47, 100,  70,  95,  18,  80,  91,  25,  92,   8,  39,  25,  29,
        84,  96,   4,  19, 111, 104,  79,  54,  97, 111,  96, 109,  19,
        15,  39,  24,  12,   6,  43, 107, 100,  13,  13,   1,  59,  71,
        91,  88,  10,  83,  32,  55,  90,  54,   5,  91,  11,  71,  62,
        21, 112,  59,  28,   9,  94,  80,  96,  57,   2,  76,  21,  15,
       105,  79, 111, 103,  76,  53,  93,  31,  10,  95,   8,  34,  18,
        60,  56,  60,  86,  16,  38,  70, 104,  34], dtype=int64)

In [27]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print("\nNext Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'rnado?\n\n118\nEarth\xe2\x80\x99s Weather\n\n\x0cWhat are other forms of\nsevere weather?\nHeavy rain is severe weather. '

Next Char Predictions:
 b'o\xe2\x96\xb2F<)\xc2\xb0Y*%`6I<G\xc2\xb0_x*it1u ?15my\x0e+\xef\xac\x81\xe2\x80\x94hNz\xef\xac\x81y\xe2\x96\xb2+\'?0$\x1eC\xe2\x80\x9d\xc2\xb0%%\x01S`tq"l8OsN\x1bt#`V-\xef\xac\x82S4!wiyQ\ne-\'\xe2\x80\x99h\xef\xac\x81\xe2\x80\x93eMv7"x :*TPTo(>_\xe2\x80\x94:'


In [29]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

tf.exp(example_batch_mean_loss).numpy()
model.compile(optimizer='adam', loss=loss)

Prediction shape:  (64, 100, 113)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.728238, shape=(), dtype=float32)


In [33]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [45]:
#history = model.fit(dataset, epochs=20, verbose=0, callbacks=[checkpoint_callback])

In [39]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        predicted_logits = predicted_logits[:, -1, :] / self.temperature
        predicted_logits += self.prediction_mask

        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
        predicted_chars = self.chars_from_ids(predicted_ids)

        return predicted_chars, states


In [41]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [43]:
start = time.time()
states = None
next_char = tf.constant(['cells'])
result = [next_char]

for n in range(100):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor([b'cells ute \n ohAear e gh e e o a  en o  nes te a a a e op\n se he s s  t o.ar gatoed  sb .\nc i h ran  l t  '], shape=(1,), dtype=string) 
________________________________________________________________________________

Run time: 2.2668282985687256
